In [11]:
import torch
import numpy as np
from typing import Iterable, Tuple, Callable
import math
from ml4gw.waveforms import SineGaussian
from ml4gw.distributions import Uniform, LogUniform, Cosine
from ml4gw.gw import compute_observed_strain

ModuleNotFoundError: No module named 'torchtyping'

One of the easiest way to improve neural network performance is to provide more high quality data. Memory limitations often reduce the amount of data that can be utilized at training time. Consider the problem of training a neural network to detect sine-Gaussian signals in gravitational wave data. In this problem, memory limitations may reduce the number of waveforms/signals available to inject into data at training time. How can we get remove this constraint? One solution is to generate these signals on GPU during training, so that memory is only utilized as it is needed. 

In [ ]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        pass

In [7]:
sample_rate = 2048 
duration = 2
ifos = ["H1", "L1"]
sine_gaussian = SineGaussian(sample_rate, duration)

NameError: name 'SineGaussian' is not defined

In [ ]:
prior = dict(
    frequency = Uniform(32, 1024),
    quality = Uniform(2, 108),
    hrss = LogUniform(1e-23, 1e-20),
    eccentricity = Uniform(0, 1),
    phase = Uniform(0, np.pi)
)

dec = Cosine()
psi = Uniform()
phi = Uniform(-math.pi, math.pi)

In [ ]:
class ParameterSampler(torch.nn.Module):
    def __init__(self, **parameters: Callable):
        self.params = parameters

    def __call__(self, N: int):
        samples = {}
        for p, dist in self.params.items():
            samples[p] = dist(N)
        return samples

parameter_sampler = ParameterSampler(**prior)

In [ ]:
parameters = parameter_sampler(1000)
cross, plus = sine_gaussian(**parameters)
projected = compute_observed_strain(dec, psi, phi, tensors, vertices)

In [ ]:
class FixedWaveformDataset:
    def __init__(
        self, 
        hanford: np.ndarray,
        livingston: np.ndarray,
        background: np.ndarray
    ):
        pass
    
    def __iter__(self):
        pass

    def __next__(self):
        pass

In [ ]:
def train_for_one_epoch(
    dataset: Iterable[Tuple[np.ndarray, np.ndarray]],
    model: torch.nn.Module,
):
    total_loss = 0
    for X, y in dataset:
        preds = model(X)
        loss = torch.nn.functional.binary_cross_entropy(preds, y)
        loss.backward()

    total_loss += loss.item()

    return total_loss


